In [3]:
import openai

In [114]:
import pandas as pd

df = pd.read_csv("synthetic_data.csv")

df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
df.drop_duplicates(inplace=True)
df = df.head()
df.head()

,Name,Address,Email,Phone Number,Date,Salary,Company,Messy Text
0,Emma Davis,"327 Broadway, New York, NY",emmadavis83@outlook.com,123.456.7890,01-02-2024,500000.0,Facebook,Can yu pleas clarrify ur qustion?
1,John Doe,"927 1st Ave, Los Angeles, CA",johndoe54@invalid,123-456-7890,"Feb 1, 2024",55000.0,Amazon Corp.,Can yu pleas clarrify ur qustion?
2,John Doe,"354 Market St, Austin, TX",johndoe44@gmail.com,123-456-7890,1st Feb 2024,500000.0,Microsoft,Can yu pleas clarrify ur qustion?
3,Emily Johnson,"112 Elm St, Los Angeles, CA",emilyjohnson14@outlook.com,(123) 456-7890,2024/02/01,150000.0,Apple,I lovee to wrk at google!
4,James White,"337 Broadway, Austin, TX",jameswhite82@outlook.com,(123) 456-7890,"Feb 1, 2024",500000.0,Amazon,tesla is a great cmpany but I wud prefer Meta.


In [75]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [ ]:
def resolve_entity(text):
    prompt = f"Resolve entity variations to a standardized form: {text}"
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

df['standardized_company'] = df['company_name'].apply(resolve_entity)


In [6]:
from dotenv import load_dotenv

load_dotenv()

import os
from openai import OpenAI
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [76]:
column_name = "company"
column_information = "name of the company"

text = "nvidia corp"
# text = "Bajaj allimc"

In [ ]:
prompt = f"""I am giving you text field from the column titled {column_name}. 
            It contains inform about {column_information} 
            Resolve it to to a standardized form but do not modify if you are not sure of the format
            for the specific data: {text}. 
            Give no preambles, just the result."""
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": prompt
        }
    ]
)

In [40]:
response.choices[0].message.content

'Nvidia Corporation'

In [32]:
response.choices[0].message.content

'Swapnil Financials'

In [46]:
def standardize_column(text, column_name, column_information):
    prompt = f"""I am giving you text field from the column titled {column_name}. 
                It contains inform about {column_information} 
                Resolve it to to a standardized form but do not modify if you are not sure of the format
                for the specific data: {text}. 
                Give no preambles, just the result."""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            }
        ]   
    )
    return response.choices[0].message.content

In [47]:
column_name = "company"
column_information = "name of the company"
df['standardized_company'] = df['Company'].apply(standardize_column, args=(column_name, column_information))

In [48]:
df['standardized_company']

0           Facebook, Inc.
1       Amazon Corporation
2    Microsoft Corporation
3               Apple Inc.
4                   Amazon
Name: standardized_company, dtype: object

In [49]:
df['Company']

0        Facebook
1    Amazon Corp.
2       Microsoft
3           Apple
4          Amazon
Name: Company, dtype: object

# Class to identify PII and mask it

In [116]:
import re
import hashlib

class PII:
    def __init__(self, df):
        self.pii_cols = None
        self.df = df
        pass

    def identify_pii(self):        
        column_name = self.df.columns.values

        prompt = f"""Here is a list of of columns present in my data: {column_name}. 
                    Your goal is to identify columns which could potentially contain PII data. 
                    From the given list of columns I shared, return the column names which could have PII. 
                    Give the result in the form of Python string (no individual quotes for strings), seperated by comma in the exact case as the original column names I gave
                    No preamble"""
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": prompt
                }
            ]   
        )
        self.pii_cols = response.choices[0].message.content.split(", ")
        return self.pii_cols
    

    def mask_pii(self):
        if self.pii_cols is None:
            raise ValueError("No PII columns identified. Run `identify_pii` first.")

        def mask_value(value, field_type):
            if pd.isna(value):  # Handle NaNs
                return value

            value = str(value)  

            if field_type == "email":
                parts = value.split("@")
                return parts[0][0] + "*****@" + parts[1] if len(parts) > 1 else "*****"

            elif field_type == "phone":
                value = re.sub(r'\D', '', value)
                return re.sub(r"(\d{2})\d{6}(\d{2})", r"\1******\2", value)  # Keep first & last 2 digits

            elif field_type in ["ssn", "passport", "credit_card"]:
                return "****-****-****-" + value[-4:]  # Keep last 4 digits

            elif field_type == "home_address":
                parts = value.split(",")
                if len(parts) >= 3:
                    return "**** " + parts[0].split()[1] + ", " + ", ".join(parts[1:])  # Mask house number
                return "**** " + value

            elif field_type == "name":
                return value[0] + "****"  # Keep first letter only

            elif field_type == "hash":
                return hashlib.sha256(value.encode()).hexdigest()  # Irreversible hashing

            else:
                return "*****"  # Default fully redact
        
        # Define common patterns to infer PII type
        column_patterns = {
            "email": ["email", "e-mail"],
            "phone": ["phone", "mobile", "contact", 'phone number'],
            "ssn": ["ssn", "social security", "social security number"],
            "passport": ["passport"],
            "credit_card": ["card number", "credit", "debit"],
            "home_address": ["address", "street", "residence"],
            "name": ["name", "full name"],
            "hash": ["token", "unique_id"]
        }

        def infer_pii_type(col_name):
            """Infer the type of PII based on column name patterns"""
            col_lower = col_name.lower()
            for pii_type, patterns in column_patterns.items():
                if any(pattern in col_lower for pattern in patterns):
                    return pii_type
            return "default"


        for col in self.pii_cols:
            pii_type = infer_pii_type(col)
            self.df[col] = self.df[col].apply(lambda x: mask_value(x, pii_type))

        return self.df
    
    def exclude_columns(self, columns):
        for column in columns:
            self.pii_cols.remove(column)   
    

In [103]:
pii = PII()
df.columns.values

array(['Name', 'Address', 'Email', 'Phone Number', 'Date', 'Salary',
       'Company', 'Messy Text'], dtype=object)

In [104]:
res = pii.identify_pii(column_name=df.columns.values)

In [105]:
res

['Name', 'Address', 'Email', 'Phone Number', 'Salary']

In [117]:
pii_processor = PII(df)
print(pii_processor.identify_pii())  # Identifies PII columns

['Name', 'Address', 'Email', 'Phone Number', 'Salary']


In [118]:
pii_processor.pii_cols

['Name', 'Address', 'Email', 'Phone Number', 'Salary']

In [119]:
pii_processor.exclude_columns(columns=['Salary'])

In [121]:
masked_df = pii_processor.mask_pii()  # Masks PII data

In [122]:
masked_df.head()

,Name,Address,Email,Phone Number,Date,Salary,Company,Messy Text
0,E****,"**** Broadway, New York, NY",e*****@outlook.com,12******90,01-02-2024,500000.0,Facebook,Can yu pleas clarrify ur qustion?
1,J****,"**** 1st, Los Angeles, CA",j*****@invalid,12******90,"Feb 1, 2024",55000.0,Amazon Corp.,Can yu pleas clarrify ur qustion?
2,J****,"**** Market, Austin, TX",j*****@gmail.com,12******90,1st Feb 2024,500000.0,Microsoft,Can yu pleas clarrify ur qustion?
3,E****,"**** Elm, Los Angeles, CA",e*****@outlook.com,12******90,2024/02/01,150000.0,Apple,I lovee to wrk at google!
4,J****,"**** Broadway, Austin, TX",j*****@outlook.com,12******90,"Feb 1, 2024",500000.0,Amazon,tesla is a great cmpany but I wud prefer Meta.


In [71]:
import pandas as pd
import re

# Sample DataFrame
df = pd.DataFrame({'phone_number': ['(123) 456-7890', '987-654-3210', '123.456.7890', '+1 (800) 555-1212']})

# Extract only digits
df['cleaned_phone_number'] = df['phone_number'].str.replace(r'\D', '', regex=True)

print(df)


        phone_number cleaned_phone_number
0     (123) 456-7890           1234567890
1       987-654-3210           9876543210
2       123.456.7890           1234567890
3  +1 (800) 555-1212          18005551212


In [113]:
df.head()

,Name,Address,Email,Phone Number,Date,Salary,Company,Messy Text
0,E****,"**** Broadway, New York, NY",e*****@outlook.com,12******90,01-02-2024,500000.0,Facebook,Can yu pleas clarrify ur qustion?
1,J****,"**** 1st, Los Angeles, CA",j*****@invalid,12******90,"Feb 1, 2024",55000.0,Amazon Corp.,Can yu pleas clarrify ur qustion?
2,J****,"**** Market, Austin, TX",j*****@gmail.com,12******90,1st Feb 2024,500000.0,Microsoft,Can yu pleas clarrify ur qustion?
3,E****,"**** Elm, Los Angeles, CA",e*****@outlook.com,12******90,2024/02/01,150000.0,Apple,I lovee to wrk at google!
4,J****,"**** Broadway, Austin, TX",j*****@outlook.com,12******90,"Feb 1, 2024",500000.0,Amazon,tesla is a great cmpany but I wud prefer Meta.
